In [ ]:
!pip install torch torchvision gradio -q

In [1]:
import torch
import numpy as np
from PIL import Image, ImageOps
import requests
import gradio as gr
from torchvision import transforms
import torchvision.models as models

In [2]:
# ---------------------------
# 1. Load Pretrained DeepLabV3
# ---------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
model = models.segmentation.deeplabv3_resnet50(pretrained=True).to(device).eval()

preprocess = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/deeplabv3_resnet50_coco-cd0a2569.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet50_coco-cd0a2569.pth


100%|██████████| 161M/161M [00:01<00:00, 99.1MB/s]


In [3]:
# ---------------------------
# 2. Segmentation Function
# ---------------------------
def get_segmentation_mask(img):
    img_t = preprocess(img).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(img_t)["out"][0]
    mask = output.argmax(0).byte().cpu().numpy()
    return mask

In [4]:
# ---------------------------
# 3. Colorization Function
# ---------------------------
def colorize(img, region="foreground"):
    img = img.convert("RGB").resize((512, 512))
    mask = get_segmentation_mask(img)

    # Foreground = non-background (class > 0)
    if region == "foreground":
        selected = mask > 0
    else:  # background
        selected = mask == 0

    # Convert to grayscale base
    gray = ImageOps.grayscale(img).convert("RGB")
    gray_np = np.array(gray)
    color_np = np.array(img)

    # Apply mask
    final = np.where(selected[..., None], color_np, gray_np)
    return Image.fromarray(final)


In [5]:
# ---------------------------
# 4. Gradio UI
# ---------------------------
demo = gr.Interface(
    fn=colorize,
    inputs=[
        gr.Image(type="pil"),
        gr.Radio(choices=["foreground", "background"], label="Region to Colorize", value="foreground")
    ],
    outputs=gr.Image(type="pil"),
    title="🎨 Targeted Colorization with Semantic Segmentation",
    description="Upload an image and choose whether to colorize the foreground or background."
)

In [ ]:
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0a3f470bbbd62372ad.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [7]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score
import numpy as np

# Evaluate segmentation mask vs ground truth (COCO-style)
# Since we don’t have GT masks in this demo, we simulate evaluation by
# treating DeepLab output as pseudo-labels and comparing against itself.
# Replace with real labels if available.

def evaluate_segmentation(pred_mask, true_mask):
    y_true = true_mask.flatten()
    y_pred = pred_mask.flatten()

    cm = confusion_matrix(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average="macro", zero_division=0)
    rec = recall_score(y_true, y_pred, average="macro", zero_division=0)
    acc = accuracy_score(y_true, y_pred)

    return cm, prec, rec, acc

# Example: run on one test image
test_img = ImageOps.grayscale(Image.open("bird2.0.jpg")).convert("RGB")
pred_mask = get_segmentation_mask(test_img)

# ⚠️ Using pred_mask as both GT and prediction (for demo)
cm, prec, rec, acc = evaluate_segmentation(pred_mask, pred_mask)

print("Confusion Matrix:\n", cm)
print("Precision:", prec)
print("Recall:", rec)
print("Accuracy:", acc * 100, "%")

Confusion Matrix:
 [[234140      0]
 [     0  28004]]
Precision: 1.0
Recall: 1.0
Accuracy: 100.0 %
